# Treasure Hunt

## Purpose
 Demonstrate probabilistic reasoning
## Author:   Rula Khayrallah
Copyright ©  Rula Khayrallah
## Warning
**Do not make any modifications to this notebook.**
## Content
In this Jupyter notebook, we have the components needed to represent and visualize a Treasure Hunt under the sea.   



## Installs and Imports

In [ ]:
!pip install import-ipynb

In [ ]:
!pip install ipycanvas

In [ ]:
import import_ipynb
import random
from utils import manhattan_distance, closest_point

from ipycanvas import MultiCanvas
from ipywidgets import Image, Label, Select, ToggleButtons, VBox, Layout, Output
out = Output()

## Main Function: _treasurehunt_

In [ ]:
def treasurehunt(size, mode):   
    if not valid_arguments(size, mode):
        return
    my_game = Game(size, mode)

def valid_arguments(size, mode):
    """
    Validate the arguments provided
    :param
     size: the number of rows/columns in the game
     mode: discovery or guided?
    :return: Boolean
    """
    valid = True
    if size <= 0:
        valid = False
        print("Invalid game size:", game_size)
    if mode not in ['discovery', 'guided']:
        valid = False
        print("Invalid mode:", mode)
        print('Modes supported: discovery, guided')
        
    return valid

## Problem Representation

In [ ]:
class Problem(object):

    """
    Problem class

    Argument:
    size (int): the number of rows/columns in the game

    Attribute:
    treasure (tuple): the location of the treasure (x, y)
    """
    def __init__(self, size):
        # Pick a random location for the treasure
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        self.treasure = x, y

    def treasure_found(self, pos):
        """
        Determine if the dive is successful
        :param pos: (tuple) diving position
        :return: boolean - True if the treasure is at that position
                 and False otherwise.
        """
        return pos == self.treasure

## Model Representation

In [ ]:
class Model(object):

    """
    Model class used to describe our sensor model

    """

    # Class variable sonar_model
    # The initial probabilities for distance <= 7
    # This is how to interpret the dictionary below:
    # P(red|d=0) = 0.7,  P(red|d=1) = 0.3, P(red|d=2) = 0.1,
    # P(red|d>=3) = 0
    # P(yellow|d=0) = 0.3, P(yellow|d=1) = 0.6, P(yellow|d=2) = 0.6, ...
    sonar_model = {'red': [0.7, 0.3, 0.1, 0],
                   'yellow': [0.3, 0.6, 0.6, 0.4, 0.3, 0.2, 0.1, 0],
                   'lime': [0, 0.1, 0.3, 0.6, 0.7, 0.8, 0.9, 1]}

    def observe(self, pos, problem):
        """
        Return the color observed when the sonar is aimed at the given
        location.
        :param pos:  (tuple) sonar position
        :param problem: (Problem object)
        :return:
        color (string) : sensor reading
        """
        dist = manhattan_distance(pos, problem.treasure)
        color = self.sample(dist)
        return color

    def sample(self, distance):
        """
        Sample a color reading based on the sensor model we have
        :param distance: (integer) Manhattan distance to the treasure
        :return: color (string) sensor reading
        """
        dist = []
        for each_color in self.sonar_model:
            farthest = len(self.sonar_model[each_color]) - 1
            if distance > farthest:
                d = farthest
            else:
                d = int(distance)
            for i in range(int(self.sonar_model[each_color][d] * 100)):
                dist.append(each_color)
        return random.choice(dist)

    def pcolorgivendist(self, color, dist):
        """
        Return the conditional probability of the given sonar
        color given the distance
        :param color: (string) sensor reading
        :param dist: (int) Manhattan distance from sensor location to a given position
        :return: float probability
        """
        farthest = len(self.sonar_model[color]) - 1
        if dist <= farthest:
            return self.sonar_model[color][dist]
        else:
            return self.sonar_model[color][farthest]

## Game Visualization

In [ ]:
class Game(object):

    """
    Game class for the treasure hunt

    Arguments:
    size (int): the number of rows/columns in the game
    mode (string): discovery or guided?  No probabilities are shown in
        discovery mode and no sensor recommendations are shown.

    Attributes
    size (int): the number of rows/columns in the game
    mode (string): discovery or guided?
    model (Model object) our world model of how the location of the
            treasure affects the sensor readings
    problem (Problem object) the specific problem (with random treasure
            location that we are trying to solve
    belief (Belief object) belief distribution based on the sensing
            evidence we have so far
    """

    square_size = 100 # length in pixels of the side of a grid square


    def __init__(self, size, mode):

        self.size = size
        self.mode = mode

        self.canvas = MultiCanvas(width=self.size * self.square_size, height=self.size * self.square_size)
        background =  self.canvas[0]
        foreground =  self.canvas[-1]

        background.line_width = 3 
        background.fill_style = 'deepskyblue'
        background.stroke_style = 'white'
        
        
        background.fill_rect(0, 0, self.size * self.square_size, self.size * self.square_size )
        
        for row in range(self.size):
            for column in range(self.size):
                 background.stroke_rect(column * self.square_size, row * self.square_size, self.square_size)

        title = Label(value = "Under the Sea Treasure Hunt", 
                      layout=Layout(display="flex", justify_content="center", border="solid"))
     
        self.action = ToggleButtons(options=['SONAR', 'DIVE'], description='', disabled=False,
                               button_style=''
                               tooltips=['Click on the SONAR button to start taking sonar measurements', 
                                         'When you are ready to make a decision and dive, click on the DIVE button'],
                               layout=Layout(display="flex", justify_content="center", width="100%", border="solid"))

    
        # load the  image files
        self.treasure_img  = Image.from_file("treasure.gif")
        self.oct = Image.from_file("octopus.gif")

            
        all_widgets = VBox(children=[title, self.action, self.canvas])
        all_widgets.layout.width = "60%"
        all_widgets.layout.height = "auto"
        display(all_widgets, out)

        self.model = Model()
        self.problem = Problem(size)
        self.belief = Belief(size)       
        if self.mode == 'guided':
            self.showbeliefs()
   
        self.canvas.on_mouse_down(self.sense)  
        self.action.observe(self.select_action)
        

    def select_action(self, change):
        """
        Set the mode to the user's selection
        """
        if change.new  == 'SONAR':
            self.canvas.on_mouse_down(self.dive, remove=True)
            self.canvas.on_mouse_down(self.sense)
            
        elif change.new == 'DIVE':
            self.canvas.on_mouse_down(self.sense, remove=True)
            self.canvas.on_mouse_down(self.dive) 
    
    
    @out.capture()  
    def dive(self, eventx, eventy):
        """
        A click at a given location represents a dive.
        Check if the dive is successful and update the GUI
        """
        row = int(eventy // self.square_size)
        column = int(eventx // self.square_size)
        self.canvas[1].clear()
        self.canvas[0].fill_style = 'black'
        self.canvas[0].fill_rect(column  * self.square_size  , row * self.square_size , self.square_size, self.square_size )
        self.canvas[0].stroke_rect(column  * self.square_size  , row * self.square_size , self.square_size, self.square_size )
 
        if self.problem.treasure_found((column,row)):
            self.canvas[0].draw_image(self.treasure_img,  (column+ 0.25) * self.square_size, (row + 0.25) * self.square_size)
        else:
            self.canvas[0].draw_image(self.oct,  (column + 0.25) * self.square_size, (row + 0.25) * self.square_size)
            if self.mode == 'guided':
                self.showbeliefs()
        

    
    @out.capture()        
    def sense(self, eventx, eventy):
        """
        A click at a given location represents a sonar reading.
        """
        row = int(eventy // self.square_size)
        column = int(eventx // self.square_size)
        sensing_position = (column, row)
        color = self.model.observe(sensing_position, self.problem)
        self.mark(row, column, color)
        self.belief.update(color, sensing_position, self.model)
        if self.mode == 'guided':
            self.canvas[1].clear()
            self.show_recommendation()
            self.showbeliefs()
            

    def showbeliefs(self):
        """
        Show the current belief distribution on the GUI
        """
        b = self.belief.current_distribution
        self.canvas[1].fill_style = 'black'
        
        self.canvas[1].text_align = 'center'
        self.canvas[1].font = "18px sans-serif"   

        for i in range(self.size):
            for j in range(self.size):
                message = f'{b[(j, i)]:4.1}'
                self.canvas[1].fill_text(message, (j + 0.5) *  self.square_size, (i + 0.5) * self.square_size)

    def show_recommendation(self):
        """
        Show the recommendation returned by the belief module in purple
        on the GUI.
        """
        next_position = self.belief.recommend_sensing()
        if next_position != NotImplemented:
            nx, ny = next_position
            self.canvas[0].fill_style = 'purple'
            self.canvas[0].fill_rect(nx * self.square_size, ny * self.square_size, self.square_size, self.square_size )
            self.canvas[0].stroke_rect(nx * self.square_size, ny * self.square_size, self.square_size, self.square_size )


    def mark(self, row, column, color):
        """
        Mark a given square with the specified color.

        :param row: (int) row of the square to be marked
        :param column: (int) column of the square to be marked
        :param color: (string)  'red', 'lime' or 'yellow'
        """
        self.canvas[0].fill_style = color
        self.canvas[0].fill_rect(column  * self.square_size  , row * self.square_size , self.square_size, self.square_size )
        self.canvas[0].stroke_rect(column  * self.square_size  , row * self.square_size , self.square_size, self.square_size )


## Belief Representation
This is a placeholder.  You will define the update and recommend methods in the beliefs module.

In [ ]:
class Belief(object):

    """
    Belief class used to track the belief distribution based on the
    sensing evidence we have so far.
    Arguments:
    size (int): the number of rows/columns in the grid

    Attributes:
    open (set of tuples): set containing all the positions that have not
        been observed so far.
    current_distribution (dictionary): probability distribution based on
        the evidence observed so far.
        The keys of the dictionary are the possible grid positions
        The values represent the (conditional) probability that the
        treasure is found at that position given the evidence
        (sensor data) observed so far.
    """

    def __init__(self, size):
        # Initially all positions are open - have not been observed
        self.open = {(x, y) for x in range(size) for y in range(size)}
        # Initialize to a uniform distribution
        self.current_distribution = {pos: 1 / (size ** 2) for pos in self.open}
        
    def update(self, color, sensor_position, model):
        pass

    def recommend_sensing(self):
        return NotImplemented